In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 3.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 829 kB 109.9 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=8815c1b0a3221c3c67e95604415534011658942da605053f8ea9fedba4c313c9
  Stored in directory: /Users/romitbarua/Library/Caches/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=4f4a96633f071d92c872ffc824bd391ce0ca20fbb1a7750f94b9639e94a0f490
  Stored in directory: /Users/romitbarua/Library/Caches/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=2137f92a4c88c1b0d4d82a6150637b3eebf2d2bd

In [1]:
import pickle

In [2]:
def buildLdaModel(corpus, dictionary, num_topics):
    
    model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    return model
    

In [2]:
def buildFeatures(df, num_topics, load_model = 0):
    
    words = df['words_for_LDA'].tolist()
    #print(words)
    dictionary = corpora.Dictionary(words)
    dictionary.filter_extremes(no_below=20, no_above=0.5)
    corpus = [dictionary.doc2bow(text) for text in words]
    
    if load_model == 0:
        model = buildLdaModel(corpus, dictionary, num_topics)

        pickle_out = open("Final_LDA_Model.pickle","wb")
        pickle.dump(model, pickle_out)
        pickle_out.close()
        
        pickle_out = open("Final_LDA_Corpus.pickle","wb")
        pickle.dump(corpus, pickle_out)
        pickle_out.close()
        
        pickle_out = open("Final_LDA_Dictionary.pickle","wb")
        pickle.dump(dictionary, pickle_out)
        pickle_out.close()
        
    else:
        pickle_in = open("Final_LDA_Model.pickle","rb")
        model = pickle.load(pickle_in)
    
    
    topics = {}
    #get a list of the topics
    for i in range(num_topics):
        topics[i] = ' '.join([term for term, freq in model.show_topic(i, topn=20)])
        
    lda_output = []
    for i in range(len(corpus)):
        probs = []
        for topic, prob in model.get_document_topics(corpus[i]):
            probs.append((topic,prob))
        lda_output.append(probs)
        
    LdaFeatures = []
    for song in lda_output:
        topic_perc = song.copy()
        song_features = []
        for num in range(num_topics):
            if topic_perc[0][0] == num:
                song_features.append(topic_perc[0][1])
                if len(topic_perc) > 1:
                    topic_perc = topic_perc[1:]
                else:
                    song_features += [0 for i in range(num+1, num_topics)]
                    break
            else:
                song_features.append(0)
        LdaFeatures.append(song_features)
        
    
    
    
    return topics, LdaFeatures
            
    
        
    